# import dependecies

In [2]:
import cv2
import os
import numpy as np
import uuid
from pathlib import Path
import json
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D

# DataSet Generation

In [10]:
def generate_dataset(name):
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    
    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        
        if len(faces) == 0:
            return None
        # Assuming single face detection for simplicity
        (x, y, w, h) = faces[0]
        cropped_face = img[y:y+h, x:x+w]
        return cropped_face
    
    cap = cv2.VideoCapture(0)
    img_id = 0
    
    while True:
        ret, frame = cap.read()
        if face_cropped(frame) is not None:
            img_id += 1
            face = cv2.resize(face_cropped(frame), (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            
            unique_filename = f"{name}_{str(uuid.uuid4())}.jpg"
            file_name_path = os.path.join("data", unique_filename)
            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            
            cv2.imshow("Cropped face", face)
        
        if cv2.waitKey(1) == 13 or img_id == 200: # 13 is the ASCII character of Enter
            break
    
    cap.release()
    cv2.destroyAllWindows()
    print("Collecting samples is completed.")

# Example usage
generate_dataset("Mahesh_Semwal")


# Save Data from SourceDir to Data and CROP

In [1]:
def crop_faces_and_save(source_dir, target_dir):
    # Initialize face cascade
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

    # Ensure the target directory exists
    os.makedirs(target_dir, exist_ok=True)

    # Process each image in the dataset
    for person_dir in Path(source_dir).iterdir():
        if person_dir.is_dir():
            person_name = person_dir.name
            for image_path in person_dir.iterdir():
                if image_path.is_file() and image_path.suffix in [".jpg", ".jpeg", ".png"]:
                    img = cv2.imread(str(image_path))
                    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                    
                    # Detect faces
                    faces = face_classifier.detectMultiScale(gray_img, scaleFactor=1.3, minNeighbors=5)
                    
                    # Crop and save each detected face
                    for i, (x, y, w, h) in enumerate(faces):
                        face = gray_img[y:y+h, x:x+w]
                        unique_filename = f"{person_name}_{i}_{str(uuid.uuid4())}.jpg"
                        file_name_path = os.path.join(target_dir, unique_filename)
                        cv2.imwrite(file_name_path, face)

    print("Face cropping and saving completed for all images in 'dataSet' directory.")


In [12]:
# Example usage:
source_dir = "dataset"
target_dir = "data"
crop_faces_and_save(source_dir, target_dir)

NameError: name 'crop_faces_and_save' is not defined

# Data Augmentation

In [29]:
# Example usage:
source_dir = "data"
target_dir = "augmented_data"

os.makedirs(target_dir, exist_ok=True)

In [30]:
def augment_and_save(image_path, target_dir):
    img = cv2.imread(image_path)
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.reshape((1,) + img.shape)
    
    # Extract the name from the original filename
    original_filename = os.path.basename(image_path)
    name = original_filename.split('_')[0]

    i = 0
    for batch in datagen.flow(img, batch_size=1, save_to_dir=target_dir, save_prefix=name, save_format='jpg'):
        i += 1
        if i > 1:
            break

    # Rename augmented files to maintain the original naming format
    for aug_file in os.listdir(target_dir):
        if aug_file.startswith(name) and not aug_file.endswith(".aug.jpg"):
            aug_uuid = uuid.uuid4()
            new_filename = f"{name}_{aug_uuid}.jpg"
            os.rename(os.path.join(target_dir, aug_file), os.path.join(target_dir, new_filename))


In [31]:
for filename in os.listdir(source_dir):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        image_path = os.path.join(source_dir, filename)
        augment_and_save(image_path, target_dir)

In [11]:
def train_classifier(data_dirs):
    if os.path.exists("label_map.json"):
        with open("label_map.json", "r") as f:
            label_map = json.load(f)
        current_id = max(label_map.values()) + 1
    else:
        label_map = {}
        current_id = 0
    
    faces = []
    labels = []

    for data_dir in data_dirs:
        path = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.jpg')]
        
        for image_path in path:
            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

            filename = os.path.split(image_path)[1]
            name = filename.split('_')[0]

            if name not in label_map:
                label_map[name] = current_id
                current_id += 1

            label = label_map[name]

            faces.append(img)
            labels.append(label)

    labels = np.array(labels)

    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, labels)
    clf.write("classifier.xml")

    with open("label_map.json", "w") as f:
        json.dump(label_map, f)

    print(f"Training completed. {len(label_map)} unique labels saved to 'label_map.json'.")

In [12]:
data_dirs = ["data"]
train_classifier(data_dirs)

Training completed. 5 unique labels saved to 'label_map.json'.


# Real-Time Face Recognition

In [4]:
def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, clf):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
     
    for (x, y, w, h) in features:
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
         
        # Perform face recognition
        id_, pred = clf.predict(gray_img[y:y + h, x:x + w])
        confidence = int(100 * (1 - pred / 300))
         
        # Display recognized name if confidence is high
        if confidence > 70:
            name = reverse_label_map.get(id_, "UNKNOWN")
            cv2.putText(img, name, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        else:
            cv2.putText(img, "UNKNOWN", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
    return img

In [5]:
def recognize(img, clf, faceCascade):
    color = {"blue": (255, 0, 0), "red": (0, 0, 255), "green": (0, 255, 0)}
    img = draw_boundary(img, faceCascade, 1.1, 10, color['red'], clf)
    return img

In [6]:
# Load the label map
import json

with open("label_map.json", "r") as f:
    label_map = json.load(f)

# Reverse the label map to get names from labels
reverse_label_map = {int(v): k for k, v in label_map.items()}

# Initialize the face cascade and the LBPH face recognizer
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
clf = cv2.face.LBPHFaceRecognizer_create()

try:
    clf.read('classifier.xml')
except Exception as e:
    print(f"Failed to read 'classifier.xml': {e}")
# Initialize the webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if ret:
        frame = recognize(frame, clf, face_cascade)
        cv2.imshow('Face Recognition', frame)

    if cv2.waitKey(1) == 13:  # 13 is the Enter key
        break

cap.release()
cv2.destroyAllWindows()